## Import thư viện

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from IPython.display import display
from pandas.plotting import scatter_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.linear_model import LinearRegression,  Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, RegressorMixin

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Overview

In [4]:
!pip install odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=95a45a527bba4fcf5ea78579d505b196dca8a13481279dcdb3458b640dfa79d3
  Stored in directory: /root/.cache/pip/wheels/c8/2e/95/90d94fe33903786937f3b8c33dd88807f792359c6424b40469
Successfully built odfpy


In [5]:
header = ['assignment_id', 'problem_id', 'username','is_final', 'status','pre_score','coefficient','language_id','created_at', 'updated_at','judgement']
df_anno = pd.read_csv('/content/drive/MyDrive/CS114/data-tbtl/annonimized.csv', names=header, header=0)
df_th = pd.read_csv('/content/drive/MyDrive/CS114/public_it001/th-public.csv')
df_tbtl = pd.read_excel('/content/drive/MyDrive/CS114/data-tbtl/tbtl-public.ods', engine='odf')
df_ck = pd.read_csv('/content/drive/MyDrive/CS114/public_it001/ck-public.csv')
df_qt = pd.read_csv('/content/drive/MyDrive/CS114/public_it001/qt-public.csv')

df_tbtl.dropna(subset=['TBTL'], inplace=True)
df_th.dropna(subset=['TH'], inplace=True)
df_ck.dropna(subset=['CK'], inplace=True)
df_qt.dropna(subset=['diemqt'], inplace=True)

In [6]:
df_anno.head()

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,judgement
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."


In [7]:
df_th.head()

,hash,TH
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7
3,013de369c439ab0ead8aa7da64423aa395a8be39,10
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,6


In [8]:
df_tbtl.head()

,username,TBTL
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.24
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.11
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.30
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,8.63
4,013de369c439ab0ead8aa7da64423aa395a8be39,8.20


In [9]:
df_ck.head()

,hash,CK
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,6.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,4.5
3,013de369c439ab0ead8aa7da64423aa395a8be39,5.0
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,4.0


In [10]:
df_qt.head()

,hash,diemqt
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7
2,01122b3ef7e59b84189e65985305f575d6bdf83c,9
3,013de369c439ab0ead8aa7da64423aa395a8be39,10
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,9


In [11]:
df_anno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295198 entries, 0 to 295197
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   assignment_id  295198 non-null  object
 1   problem_id     295198 non-null  object
 2   username       295198 non-null  object
 3   is_final       295198 non-null  int64 
 4   status         295198 non-null  object
 5   pre_score      295198 non-null  int64 
 6   coefficient    295198 non-null  int64 
 7   language_id    295198 non-null  object
 8   created_at     295198 non-null  object
 9   updated_at     295198 non-null  object
 10  judgement      295198 non-null  object
dtypes: int64(3), object(8)
memory usage: 24.8+ MB


In [12]:
df_tbtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   username  799 non-null    object 
 1   TBTL      799 non-null    float64
dtypes: float64(1), object(1)
memory usage: 12.6+ KB


In [13]:
df_th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 755 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hash    755 non-null    object
 1   TH      755 non-null    object
dtypes: object(2)
memory usage: 17.7+ KB


In [14]:
df_qt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 755 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hash    755 non-null    object
 1   diemqt  755 non-null    object
dtypes: object(2)
memory usage: 17.7+ KB


In [15]:
df_ck.info()

<class 'pandas.core.frame.DataFrame'>
Index: 755 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hash    755 non-null    object 
 1   CK      755 non-null    float64
dtypes: float64(1), object(1)
memory usage: 17.7+ KB


In [16]:
#Đổi tên các cột df_tbtl, df_th, df_qt, df_ck để đồng bộ hơn:
df_tbtl.rename(columns={"username" : "mssv"}, inplace=True)
df_th.rename(columns={"hash" : "mssv"}, inplace=True)
df_qt.rename(columns={"hash" : "mssv"}, inplace=True)
df_ck.rename(columns={"hash" : "mssv"}, inplace=True)

# Chuyển đổi cột điểm ở df_th, df_qt sang dạng số, bỏ qua các giá trị không phải số
df_th['TH'] = pd.to_numeric(df_th['TH'], errors='coerce')
df_qt['diemqt'] = pd.to_numeric(df_qt['diemqt'], errors='coerce')

In [17]:
df_th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 755 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mssv    755 non-null    object 
 1   TH      753 non-null    float64
dtypes: float64(1), object(1)
memory usage: 17.7+ KB


In [18]:
df_qt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 755 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mssv    755 non-null    object 
 1   diemqt  754 non-null    float64
dtypes: float64(1), object(1)
memory usage: 17.7+ KB


## Feature Engineering

In [19]:
# Lọc ra những bài nộp cuối cùng và đúng của mỗi sinh viên
successess_submissions = df_anno[(df_anno['is_final'] == 1) & (df_anno['pre_score'] == 10000)]
# Nhóm theo 'username' và đếm số problem đúng, số lần submit đúng của mỗi sinh viên
prob_successess_count = successess_submissions.groupby('username')['problem_id'].nunique().reset_index(name='num_prob_success')
sub_successess_count = successess_submissions.groupby('username').size().reset_index(name='num_submit_success')
# Gộp kết quả trên vào df
df_anno = pd.merge(df_anno, prob_successess_count, on='username', how='left')
df_anno = pd.merge(df_anno, sub_successess_count, on='username', how='left')
# Điền 0 vào các vị trí NaN (những sinh viên không có bài nào)
df_anno['num_prob_success'] = df_anno['num_prob_success'].fillna(0).astype(int)
df_anno['num_submit_success'] = df_anno['num_submit_success'].fillna(0).astype(int)
df_anno

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,judgement,num_prob_success,num_submit_success
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:03:43,01-15 16:03:53,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",96,96
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,01-15 16:04:07,01-15 16:05:08,"{""times"":[],""mems"":[],""verdicts"":{""\n\nIn func...",96,96
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:04:58,01-15 16:05:08,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",96,96
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:05:13,01-15 16:05:22,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":[0...",96,96


In [20]:
# Số bài đã nộp
submission_counts = df_anno.groupby('username').size()
df_anno['num_submit'] = df_anno['username'].map(submission_counts)
# Số assignment đã thử
trial_asgmt_counts = df_anno.groupby('username')['assignment_id'].nunique()
df_anno['num_trial_asgmt'] = df_anno['username'].map(trial_asgmt_counts)
# Số problem dã thử
trial_prob_counts = df_anno.groupby('username')['problem_id'].nunique()
df_anno['num_trial_prob'] = df_anno['username'].map(trial_prob_counts)
df_anno

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,judgement,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:03:43,01-15 16:03:53,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",96,96,228,11,98
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,01-15 16:04:07,01-15 16:05:08,"{""times"":[],""mems"":[],""verdicts"":{""\n\nIn func...",96,96,228,11,98
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:04:58,01-15 16:05:08,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",96,96,228,11,98
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:05:13,01-15 16:05:22,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":[0...",96,96,228,11,98


In [21]:
successess_submit = df_anno[(df_anno['is_final'] == 1) & (df_anno['pre_score'] == 10000)]
successess_submit = successess_submit.drop_duplicates(subset=['username', 'problem_id'], keep='first')
# Tính điểm cho mỗi problem_id đã pass
successess_submit['score'] = (successess_submit['pre_score']/100) * (successess_submit['coefficient']/100)
# Tính tổng điểm cho mỗi mssv
total_scores = successess_submit.groupby('username')['score'].sum().reset_index(name='total_score')
# Gán tổng điểm vào df gốc
df_anno = df_anno.merge(total_scores, on='username', how='left')
# Điền các giá trị NaN bằng 0 nếu sinh viên không có bài nào pass
df_anno['total_score'] = df_anno['total_score'].fillna(0)
display(df_anno)

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,judgement,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58,5600.0
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58,5600.0
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58,5600.0
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58,5600.0
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0...",56,56,139,5,58,5600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:03:43,01-15 16:03:53,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",96,96,228,11,98,9600.0
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,01-15 16:04:07,01-15 16:05:08,"{""times"":[],""mems"":[],""verdicts"":{""\n\nIn func...",96,96,228,11,98,9600.0
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:04:58,01-15 16:05:08,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":...",96,96,228,11,98,9600.0
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,01-15 16:05:13,01-15 16:05:22,"{""times"":[0,0,0,0,0,0,0,0,0,0,0,0,0],""mems"":[0...",96,96,228,11,98,9600.0


In [22]:
# Định dạng lại thời gian
df_anno['created_at'] = pd.to_datetime('2024-' + df_anno['created_at'], format='%Y-%m-%d %H:%M:%S')

# Thêm các cột số baì nộp theo tháng của sinh viên
df_anno['month'] = df_anno['created_at'].dt.month
month_dummies = pd.get_dummies(df_anno['month'], prefix='month')
df_anno = pd.concat([df_anno, month_dummies], axis=1)

# Tính tổng số bài nộp theo tháng của mỗi sinh viên
for column in month_dummies.columns:
    df_anno[column] = df_anno.groupby('username')[column].transform('sum')

# Phân loại dữ liệu theo các khoảng giờ
df_anno['hour'] = df_anno['created_at'].dt.hour
bins = [0, 3, 6, 9, 12, 15, 18, 21, 24]
labels = ['0-2', '3-5', '6-8', '9-11', '12-14', '15-17', '18-20', '21-23']
df_anno['time_slot'] = pd.cut(df_anno['hour'], bins=bins, labels=labels, right=True)
# Tạo các cột mới cho mỗi khoảng giờ và tính số lượng bài nộp
for label in labels:
    df_anno[label] = (df_anno['time_slot'] == label).astype(int)
    df_anno[label] = df_anno.groupby('username')[label].transform('sum')
# Xoá các cột không cần thiết
df_anno = df_anno.drop(columns=['hour','time_slot'])

df_anno

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,...,month_11,month_12,0-2,3-5,6-8,9-11,12-14,15-17,18-20,21-23
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:02:04,10-09 08:06:58,...,31,0,8,20,47,26,17,15,0,0
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:04:41,10-09 08:04:51,...,31,0,8,20,47,26,17,15,0,0
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:06:49,10-09 08:06:58,...,31,0,8,20,47,26,17,15,0,0
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:47:52,10-09 08:48:01,...,31,0,8,20,47,26,17,15,0,0
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 09:19:35,10-09 09:19:45,...,31,0,8,20,47,26,17,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:03:43,01-15 16:03:53,...,69,39,17,14,49,28,73,34,12,1
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,2024-01-15 16:04:07,01-15 16:05:08,...,69,39,17,14,49,28,73,34,12,1
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:04:58,01-15 16:05:08,...,69,39,17,14,49,28,73,34,12,1
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:05:13,01-15 16:05:22,...,69,39,17,14,49,28,73,34,12,1


In [23]:
status_dummies = pd.get_dummies(df_anno['status'], prefix='status_')
df_status = pd.concat([df_anno, status_dummies], axis=1)
status_counts = df_status.groupby('username')[status_dummies.columns].sum().reset_index()
df_anno = df_anno.merge(status_counts, on='username', how='left')
df_anno

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,...,6-8,9-11,12-14,15-17,18-20,21-23,status__Compilation Error,status__SCORE,status__Syntax Error,status__pending
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:02:04,10-09 08:06:58,...,47,26,17,15,0,0,34,105,0,0
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:04:41,10-09 08:04:51,...,47,26,17,15,0,0,34,105,0,0
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:06:49,10-09 08:06:58,...,47,26,17,15,0,0,34,105,0,0
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:47:52,10-09 08:48:01,...,47,26,17,15,0,0,34,105,0,0
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 09:19:35,10-09 09:19:45,...,47,26,17,15,0,0,34,105,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:03:43,01-15 16:03:53,...,49,28,73,34,12,1,45,183,0,0
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,2024-01-15 16:04:07,01-15 16:05:08,...,49,28,73,34,12,1,45,183,0,0
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:04:58,01-15 16:05:08,...,49,28,73,34,12,1,45,183,0,0
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:05:13,01-15 16:05:22,...,49,28,73,34,12,1,45,183,0,0


In [24]:
# Tính tỷ lệ phần trăm của số vấn đề được giải so với tổng số lần nộp
df_anno['prob_success_ratio'] = df_anno['num_prob_success'] / df_anno['num_submit'] * 100

# Tính tỷ lệ phần trăm của số lần nộp giải được so với tổng số lần nộp
df_anno['sub_success_ratio'] = df_anno['num_submit_success'] / df_anno['num_submit'] * 100

# Tính tỷ lệ phần trăm lỗi
df_anno['error_ratio'] = (1 - df_anno['status__SCORE']/df_anno['num_submit']) * 100

# Tính tỷ lệ phần trăm sinh viên nộp bài trong khoảng từ 0h đến 5h
df_anno['soon_submit_ratio'] = (df_anno['0-2']+df_anno['3-5'])/df_anno['num_submit'] * 100
df_anno

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,...,18-20,21-23,status__Compilation Error,status__SCORE,status__Syntax Error,status__pending,prob_success_ratio,sub_success_ratio,error_ratio,soon_submit_ratio
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:02:04,10-09 08:06:58,...,0,0,34,105,0,0,40.287770,40.287770,24.460432,20.143885
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:04:41,10-09 08:04:51,...,0,0,34,105,0,0,40.287770,40.287770,24.460432,20.143885
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:06:49,10-09 08:06:58,...,0,0,34,105,0,0,40.287770,40.287770,24.460432,20.143885
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:47:52,10-09 08:48:01,...,0,0,34,105,0,0,40.287770,40.287770,24.460432,20.143885
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 09:19:35,10-09 09:19:45,...,0,0,34,105,0,0,40.287770,40.287770,24.460432,20.143885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:03:43,01-15 16:03:53,...,12,1,45,183,0,0,42.105263,42.105263,19.736842,13.596491
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,2024-01-15 16:04:07,01-15 16:05:08,...,12,1,45,183,0,0,42.105263,42.105263,19.736842,13.596491
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:04:58,01-15 16:05:08,...,12,1,45,183,0,0,42.105263,42.105263,19.736842,13.596491
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:05:13,01-15 16:05:22,...,12,1,45,183,0,0,42.105263,42.105263,19.736842,13.596491


In [25]:
#Xác định các assignment_id cho mỗi problem_id
assignments_per_problem = df_anno.groupby('problem_id')['assignment_id'].unique()

#Tính số lượng sinh viên đã tham gia mỗi assignment
students_per_assignment = df_anno.groupby('assignment_id')['username'].nunique()

#Xác định số sinh viên đã tham gia mỗi problem_id
students_per_problem = assignments_per_problem.apply(lambda x: students_per_assignment.loc[x].sum())

#Tính số sinh viên đã giải được mỗi problem_id
final_solutions = df_anno[(df_anno['is_final'] == 1) & (df_anno['pre_score'] == 10000)]
solved_counts = final_solutions.groupby('problem_id')['username'].nunique()

#Tính tỷ lệ giữa số sinh viên đã giải được và số sinh viên đã tham gia
solved_ratio = (solved_counts / students_per_problem * 100).fillna(0)

In [26]:
def classify_ratio(ratio):
    if ratio >= 60:
        return 'level_1'
    elif 35 <= ratio < 60:
        return 'level_2'
    elif 20 <= ratio < 35:
        return 'level_3'
    elif 10 <= ratio < 20:
        return 'level_4'
    else:
        return 'level_5'

lv = solved_ratio.apply(classify_ratio).reset_index(name='level')

In [27]:
# Ghép cột level vào df gốc
df_anno = pd.merge(df_anno, lv, on='problem_id', how='left')

# Lọc các bài nộp thành công
success_submissions = df_anno[(df_anno['is_final'] == 1) & (df_anno['pre_score'] == 10000)]

# Nhóm theo username và level, đếm số lượng
level_counts = success_submissions.groupby(['username', 'level']).size().unstack(fill_value=0).reset_index()

# Gộp vào df gốc
df_anno = pd.merge(df_anno, level_counts, on='username', how='left')

# Điền 0 vào các vị trí NaN
level_columns = ['level_1', 'level_2', 'level_3', 'level_4', 'level_5']
for column in level_columns:
    df_anno[column] = df_anno[column].fillna(0).astype(int)

# Xoá cột level
df_anno.drop(columns='level', inplace=True)

In [28]:
df_anno

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,...,status__pending,prob_success_ratio,sub_success_ratio,error_ratio,soon_submit_ratio,level_1,level_2,level_3,level_4,level_5
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:02:04,10-09 08:06:58,...,0,40.287770,40.287770,24.460432,20.143885,54,2,0,0,0
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:04:41,10-09 08:04:51,...,0,40.287770,40.287770,24.460432,20.143885,54,2,0,0,0
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:06:49,10-09 08:06:58,...,0,40.287770,40.287770,24.460432,20.143885,54,2,0,0,0
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:47:52,10-09 08:48:01,...,0,40.287770,40.287770,24.460432,20.143885,54,2,0,0,0
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 09:19:35,10-09 09:19:45,...,0,40.287770,40.287770,24.460432,20.143885,54,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:03:43,01-15 16:03:53,...,0,42.105263,42.105263,19.736842,13.596491,87,9,0,0,0
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,2024-01-15 16:04:07,01-15 16:05:08,...,0,42.105263,42.105263,19.736842,13.596491,87,9,0,0,0
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:04:58,01-15 16:05:08,...,0,42.105263,42.105263,19.736842,13.596491,87,9,0,0,0
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:05:13,01-15 16:05:22,...,0,42.105263,42.105263,19.736842,13.596491,87,9,0,0,0


Mã hoá các cột problem_id, languge_id, assignment_id bằng Onehot-encoding

In [29]:
encoder = OneHotEncoder(sparse_output=False)
df_anno['problem_id'] = df_anno['problem_id'].str.strip()

# Thực hiện fit_transform trên cột 'problem_id'
encoded_columns = encoder.fit_transform(df_anno[['problem_id']])

# Lấy tên các cột sau khi encoding
encoded_column_names = [cat for cat in encoder.categories_[0]]

# Tạo DataFrame cho các cột đã được One-Hot Encoded
encoded_df = pd.DataFrame(encoded_columns, columns=encoded_column_names)

# Kết hợp DataFrame ban đầu với các cột mới
df_anno = pd.concat([df_anno, encoded_df], axis=1)

encoder2 = OneHotEncoder(sparse_output=False)
df_anno['language_id'] = df_anno['language_id'].str.strip()

# Thực hiện fit_transform trên cột 'languge_id'
encoded2_columns = encoder2.fit_transform(df_anno[['language_id']])

# Lấy tên các cột sau khi encoding
encoded2_column_names = [cat for cat in encoder2.categories_[0]]

# Tạo DataFrame cho các cột đã được One-Hot Encoded
encoded2_df = pd.DataFrame(encoded2_columns, columns=encoded2_column_names)

# Kết hợp DataFrame ban đầu với các cột mới
df_anno = pd.concat([df_anno, encoded2_df], axis=1)

encoder3 = OneHotEncoder(sparse_output=False)
df_anno['assignment_id'] = df_anno['assignment_id'].str.strip()

# Thực hiện fit_transform trên cột 'assignment_id'
encoded3_columns = encoder3.fit_transform(df_anno[['assignment_id']])

# Lấy tên các cột sau khi encoding
encoded3_column_names = [cat for cat in encoder3.categories_[0]]

# Tạo DataFrame cho các cột đã được One-Hot Encoded
encoded3_df = pd.DataFrame(encoded3_columns, columns=encoded3_column_names)

# Kết hợp DataFrame ban đầu với các cột mới
df_anno = pd.concat([df_anno, encoded3_df], axis=1)
display(df_anno)

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,created_at,updated_at,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:02:04,10-09 08:06:58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2024-10-09 08:04:41,10-09 08:04:51,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:06:49,10-09 08:06:58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 08:47:52,10-09 08:48:01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2024-10-09 09:19:35,10-09 09:19:45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295193,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,388516cbf597351226be1bdbe5ef30b9dcef570f,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:03:43,01-15 16:03:53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295194,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,0,Compilation Error,0,100,it0012,2024-01-15 16:04:07,01-15 16:05:08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295195,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,d2b96124ccb8e27b4b8dacdb935e729cb1ba546b,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:04:58,01-15 16:05:08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295196,613aea04c978f5e72fffc8bcff1f7b695a63f7b1,8c0f8dd4ff55e1609f733e043ac5e88b1dde6e7c,232cce96362898f08e9150ba244adaf2d6583ab2,1,SCORE,10000,100,it0012,2024-01-15 16:05:13,01-15 16:05:22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
def create_train_predict_df(df, df_anno):
    train_df = df.copy()
    user_id = train_df['mssv']
    user_anno_id = df_anno['username']

    # Lọc ra những sinh viên có trong anno_df mà không có trong df đã có kết quả
    unique_user_id = user_anno_id[~user_anno_id.isin(user_id)]

    # Lọc chỉ giữ lại các hàng của sinh viên có trong danh sách trên
    fil_df = df_anno[df_anno['username'].isin(unique_user_id)]

    # Lấy ra danh sách các sinh viên
    list_username = fil_df['username'].unique()
    predict_df = pd.DataFrame(list_username, columns=['mssv'])
    predict_col = df.columns[1]

    # Tạo train và predict dataset
    first_user_data = df_anno.groupby('username').first().reset_index()
    train_df = pd.merge(train_df, first_user_data, left_on='mssv', right_on='username', how='left')
    predict_df = pd.merge(predict_df, first_user_data, left_on='mssv', right_on='username', how='left')

    # Xoá các cột không cần thiết
    drop_col = ['username', 'assignment_id', 'problem_id', 'is_final', 'status', 'pre_score', 'coefficient', 'language_id', 'created_at', 'updated_at', 'judgement']
    train_df.drop(columns=drop_col, inplace=True)
    predict_df.drop(columns=drop_col, inplace=True)
    train_df.dropna(subset=predict_col, inplace=True)
    return train_df, predict_df

train_df_TH, predict_df_TH = create_train_predict_df(df_th, df_anno)
train_df_QT, predict_df_QT = create_train_predict_df(df_qt, df_anno)
train_df_CK, predict_df_CK = create_train_predict_df(df_ck, df_anno)
train_df_TBTL, predict_df_TBTL = create_train_predict_df(df_tbtl, df_anno)

#### TH

In [31]:
train_df_TH

,mssv,TH,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.0,45,46,147,7,46,4500.0,9,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.5,72,72,259,9,78,7200.0,9,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.0,58,59,195,7,66,5800.0,9,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,013de369c439ab0ead8aa7da64423aa395a8be39,10.0,44,58,107,8,52,4400.0,10,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,6.0,79,79,199,9,90,7900.0,10,26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,840d07858c03f80f4695056e2cc7d0c474b83a25,6.0,37,39,111,6,43,3700.0,9,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
751,844f5db2e7e31ae51eba025480679ed7e4708ac6,8.5,102,102,357,17,104,10200.0,9,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752,845acd04a77b3d1b623f255d9f9f8eae90892dab,5.5,47,47,155,8,52,4700.0,9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
753,8460eaaf887a6289fb156f7562fb739ba8e9629e,10.0,109,109,557,12,114,10900.0,9,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
predict_df_TH

,mssv,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,month_2,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,56,56,139,5,58,5600.0,10,48,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ba12c0a2cb367af0467e479c03507c71a805d291,112,117,319,6,112,11200.0,9,95,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,b7298b0fe50443a623af9b56792b330c2d052845,108,108,247,5,110,10800.0,9,98,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c60be70309789b39355dc612f36e37090ccad5dc,76,76,172,10,83,7600.0,9,40,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a22a58c5be8aa2c2700619e37f2b7a6e4efa7e6b,98,98,288,11,103,9800.0,9,28,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,508e0627871ed482bd8ab34544e43e5d856a889c,33,33,94,4,33,3300.0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
730,d525d6edf4b0c6362ddaa4814c3930a6a62bdc99,97,97,217,10,98,9700.0,9,22,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
731,b722e6209f2858faf0bf80947cadcbde586bb666,22,22,136,3,26,2200.0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732,b45e8d507778dab56f381a681c453cbf4d3b4050,45,45,141,10,53,4500.0,9,35,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### QT

In [33]:
train_df_QT

,mssv,diemqt,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.5,45,46,147,7,46,4500.0,9,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7.0,72,72,259,9,78,7200.0,9,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01122b3ef7e59b84189e65985305f575d6bdf83c,9.0,58,59,195,7,66,5800.0,9,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,013de369c439ab0ead8aa7da64423aa395a8be39,10.0,44,58,107,8,52,4400.0,10,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,9.0,79,79,199,9,90,7900.0,10,26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,840d07858c03f80f4695056e2cc7d0c474b83a25,9.0,37,39,111,6,43,3700.0,9,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
751,844f5db2e7e31ae51eba025480679ed7e4708ac6,8.0,102,102,357,17,104,10200.0,9,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.5,47,47,155,8,52,4700.0,9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
753,8460eaaf887a6289fb156f7562fb739ba8e9629e,10.0,109,109,557,12,114,10900.0,9,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
predict_df_QT

,mssv,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,month_2,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,56,56,139,5,58,5600.0,10,48,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ba12c0a2cb367af0467e479c03507c71a805d291,112,117,319,6,112,11200.0,9,95,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,b7298b0fe50443a623af9b56792b330c2d052845,108,108,247,5,110,10800.0,9,98,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c60be70309789b39355dc612f36e37090ccad5dc,76,76,172,10,83,7600.0,9,40,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a22a58c5be8aa2c2700619e37f2b7a6e4efa7e6b,98,98,288,11,103,9800.0,9,28,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,508e0627871ed482bd8ab34544e43e5d856a889c,33,33,94,4,33,3300.0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
730,d525d6edf4b0c6362ddaa4814c3930a6a62bdc99,97,97,217,10,98,9700.0,9,22,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
731,b722e6209f2858faf0bf80947cadcbde586bb666,22,22,136,3,26,2200.0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732,b45e8d507778dab56f381a681c453cbf4d3b4050,45,45,141,10,53,4500.0,9,35,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### CK

In [35]:
train_df_CK

,mssv,CK,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.5,45,46,147,7,46,4500.0,9,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,6.5,72,72,259,9,78,7200.0,9,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01122b3ef7e59b84189e65985305f575d6bdf83c,4.5,58,59,195,7,66,5800.0,9,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,013de369c439ab0ead8aa7da64423aa395a8be39,5.0,44,58,107,8,52,4400.0,10,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,4.0,79,79,199,9,90,7900.0,10,26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,840d07858c03f80f4695056e2cc7d0c474b83a25,5.5,37,39,111,6,43,3700.0,9,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
751,844f5db2e7e31ae51eba025480679ed7e4708ac6,6.0,102,102,357,17,104,10200.0,9,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
752,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.0,47,47,155,8,52,4700.0,9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
753,8460eaaf887a6289fb156f7562fb739ba8e9629e,7.0,109,109,557,12,114,10900.0,9,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
predict_df_CK

,mssv,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,month_2,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,56,56,139,5,58,5600.0,10,48,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ba12c0a2cb367af0467e479c03507c71a805d291,112,117,319,6,112,11200.0,9,95,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,b7298b0fe50443a623af9b56792b330c2d052845,108,108,247,5,110,10800.0,9,98,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c60be70309789b39355dc612f36e37090ccad5dc,76,76,172,10,83,7600.0,9,40,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a22a58c5be8aa2c2700619e37f2b7a6e4efa7e6b,98,98,288,11,103,9800.0,9,28,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,508e0627871ed482bd8ab34544e43e5d856a889c,33,33,94,4,33,3300.0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
730,d525d6edf4b0c6362ddaa4814c3930a6a62bdc99,97,97,217,10,98,9700.0,9,22,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
731,b722e6209f2858faf0bf80947cadcbde586bb666,22,22,136,3,26,2200.0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732,b45e8d507778dab56f381a681c453cbf4d3b4050,45,45,141,10,53,4500.0,9,35,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### TBTL

In [37]:
train_df_TBTL

,mssv,TBTL,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.24,45,46,147,7,46,4500.0,9,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.11,72,72,259,9,78,7200.0,9,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.30,58,59,195,7,66,5800.0,9,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,8.63,47,47,100,4,47,4700.0,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,013de369c439ab0ead8aa7da64423aa395a8be39,8.20,44,58,107,8,52,4400.0,10,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,840d07858c03f80f4695056e2cc7d0c474b83a25,7.67,37,39,111,6,43,3700.0,9,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
795,844f5db2e7e31ae51eba025480679ed7e4708ac6,7.54,102,102,357,17,104,10200.0,9,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
796,845acd04a77b3d1b623f255d9f9f8eae90892dab,7.49,47,47,155,8,52,4700.0,9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,8460eaaf887a6289fb156f7562fb739ba8e9629e,8.94,109,109,557,12,114,10900.0,9,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
predict_df_TBTL

,mssv,num_prob_success,num_submit_success,num_submit,num_trial_asgmt,num_trial_prob,total_score,month,month_1,month_2,...,f86c876f6d60b3b0c14fb0f084c705f1c4bb3dad,f89d30fccecbe52278b59e9294d868e3c51fbff4,f8d46b863c7cb8ec934fa0043264dfdb01209630,fa76cedef89934b5a8c9b6f93744b2cbdd941e6d,fd8fdf743e38d0dc3e613563f3ce1ad0301235bb,fdb39b60bce4591bdac572cc1f03d4f3417df16e,fe1bf6b299645a8e2502b0a43fa57d650d1b1b41,feac75b57862fb1653d3835bea439742ea93c8c8,febe93930de7a9a910e8a55177a7bce45de7329b,ff1bb207f10b6d6999751f65f3abf047f2b9242c
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,56,56,139,5,58,5600.0,10,48,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ba12c0a2cb367af0467e479c03507c71a805d291,112,117,319,6,112,11200.0,9,95,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,b7298b0fe50443a623af9b56792b330c2d052845,108,108,247,5,110,10800.0,9,98,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c60be70309789b39355dc612f36e37090ccad5dc,76,76,172,10,83,7600.0,9,40,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a22a58c5be8aa2c2700619e37f2b7a6e4efa7e6b,98,98,288,11,103,9800.0,9,28,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,f0d74f349e6479eaecf425634f8b0dd51e60d60c,54,54,229,6,61,5400.0,10,70,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
686,d525d6edf4b0c6362ddaa4814c3930a6a62bdc99,97,97,217,10,98,9700.0,9,22,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
687,b722e6209f2858faf0bf80947cadcbde586bb666,22,22,136,3,26,2200.0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
688,b45e8d507778dab56f381a681c453cbf4d3b4050,45,45,141,10,53,4500.0,9,35,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
print(train_df_TBTL.columns[1])

TBTL


## Train model

In [40]:
def search_model(X, y, models):
    best_models = []
    for i, (name, model, params) in enumerate(models, start=1):
        # Tạo pipeline
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('regressor', model)
        ])

        # Thiết lập lưới tham số
        param_grid = {
            'scaler': [StandardScaler(), MinMaxScaler(), None],
            **params
        }

        # Thực hiện GridSearchCV
        kf = KFold(n_splits=5, shuffle=True, random_state=123456)
        grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='r2', n_jobs=12, verbose = 1)
        grid_search.fit(X, y)

        # Tính toán điểm đáng giá chéo
        cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=kf, scoring='r2')
        mean_cv_score = np.mean(cv_scores)

        # In ra kết quả
        print(f"#{i} Model: {name}")
        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Mean cross-validation score: {mean_cv_score}")

        # Lưu mô hình tốt nhất
        best_models.append(grid_search.best_estimator_)
        print('-' * 50)

    return best_models

In [41]:
class WeightedVotingEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights

    def predict(self, X):
        predict = np.array([model.predict(X) for model in self.models])
        mean_predict = np.average(predict, axis=0, weights=self.weights)
        return mean_predict

In [42]:
models = [

        ('knn_regressor', KNeighborsRegressor(), {
            'regressor__n_neighbors': [3, 5, 10],
            'regressor__weights': ['uniform', 'distance']
        }),
        ('random_forest', RandomForestRegressor(random_state=123456), {
            'regressor__n_estimators': [50, 70, 100],
            'regressor__max_depth': [None, 3, 5, 7, 10]
        }),
        ('svr', SVR(), {
            'regressor__C': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0 ,7.0],
            'regressor__kernel': ['rbf'],
            'regressor__epsilon': [0.2, 0.4, 0.6, 0.8, 1]
        }),
        ('ridge', Ridge(), {
            'regressor__alpha': [0.5, 1, 1.5, 2, 2.5, 3, 4, 5]
        }),
        ('lasso', Lasso(max_iter=500000), {
            'regressor__alpha':  [0.2, 0.4, 0.6, 0.8, 1]
        }),
        ('elastic_net', ElasticNet(max_iter=500000), {
            'regressor__alpha': [0.1, 0.3, 0.5, 1, 1.5, 2, 2.5, 3],
            'regressor__l1_ratio': [0.2, 0.4, 0.6, 0.8, 1]
        }),
        ('adaboost', AdaBoostRegressor(random_state=123456), {
            'regressor__n_estimators': [10, 30, 50, 100, 200, 300],
            'regressor__learning_rate': [0.01, 0.03, 0.1]
        }),
        ('xgboost', xgb.XGBRegressor(), {
            'regressor__n_estimators': [100, 200, 300],
            'regressor__max_depth': [3, 5, 7],
            'regressor__learning_rate': [0.01, 0.1, 0.2]
        })
    ]

Tìm mô hình và tham số tốt nhất cho việc dự đoán điểm TBTL

In [ ]:
X_TBTL = train_df_TBTL.drop(columns=['mssv', 'TBTL'])
y_TBTL = train_df_TBTL['TBTL']
search_model(X_TBTL, y_TBTL, models)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
#1 Model: knn_regressor
Best parameters: {'regressor__n_neighbors': 10, 'regressor__weights': 'uniform', 'scaler': MinMaxScaler()}
Mean cross-validation score: 0.15396022364946926
--------------------------------------------------
Fitting 5 folds for each of 45 candidates, totalling 225 fits
#2 Model: random_forest
Best parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 100, 'scaler': StandardScaler()}
Mean cross-validation score: 0.1580104036220042
--------------------------------------------------
Fitting 5 folds for each of 105 candidates, totalling 525 fits
#3 Model: svr
Best parameters: {'regressor__C': 2.0, 'regressor__epsilon': 0.4, 'regressor__kernel': 'rbf', 'scaler': MinMaxScaler()}
Mean cross-validation score: 0.23741747798900245
--------------------------------------------------
Fitting 5 folds for each of 24 candidates, totalling 120 fits
#4 Model: ridge
Best parameters: {'regressor__alpha': 5,

In [ ]:
model_TBTL1 = Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=5))])
model_TBTL1.fit(X_TBTL, y_TBTL)

model_TBTL2 = Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=2.0, epsilon=0.4))])
model_TBTL2.fit(X_TBTL, y_TBTL)

list_model_TBTL = [model_TBTL1, model_TBTL2]
list_weight_TBTL = [0.5, 0.5]
ensemble_model_TBTL = WeightedVotingEnsemble(models=list_model_TBTL, weights=list_weight_TBTL)

Tìm mô hình và tham số tốt nhất cho việc dự đoán điểm quá trình

In [ ]:
X_QT = train_df_QT.drop(columns=['mssv', 'diemqt'])
y_QT = train_df_QT['diemqt']
search_model(X_QT, y_QT, models)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
#1 Model: linear
Best parameters: {'scaler': MinMaxScaler()}
Mean cross-validation score: 0.21485838137612573
------------------------------
Fitting 5 folds for each of 24 candidates, totalling 120 fits
#2 Model: ridge
Best parameters: {'regressor__alpha': 3, 'scaler': MinMaxScaler()}
Mean cross-validation score: 0.3392460077975472
------------------------------
Fitting 5 folds for each of 15 candidates, totalling 75 fits
#3 Model: lasso
Best parameters: {'regressor__alpha': 0.2, 'scaler': StandardScaler()}
Mean cross-validation score: 0.21672581232102878
------------------------------
Fitting 5 folds for each of 120 candidates, totalling 600 fits
#4 Model: elastic_net
Best parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.4, 'scaler': StandardScaler()}
Mean cross-validation score: 0.3416819247001839
------------------------------
Fitting 5 folds for each of 105 candidates, totalling 525 fits
#5 Model: svr
Best pa

[Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', LinearRegression())]),
 Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=3))]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor', Lasso(alpha=0.2, max_iter=500000))]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor',
                  ElasticNet(alpha=0.1, l1_ratio=0.4, max_iter=500000))]),
 Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=5.0, epsilon=0.4))]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor',
                  AdaBoostRegressor(learning_rate=0.03, random_state=123456))])]

In [ ]:
model_QT1 = Pipeline(steps=[('scaler', MinMaxScaler()),
                            ('regressor', SVR(C=5.0, epsilon=0.4))])
model_QT1.fit(X_QT, y_QT)

model_QT2 = Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor', ElasticNet(alpha=0.1, l1_ratio=0.4, max_iter=100000))])
model_QT2.fit(X_QT, y_QT)

list_model_QT = [model_QT1, model_QT2]
list_weight_QT = [0.5, 0.5]
ensemble_model_QT = WeightedVotingEnsemble(models=list_model_QT, weights=list_weight_QT)

Tìm mô hình và tham số tốt nhất cho việc dự đoán điểm thực hành

In [ ]:
X_TH = train_df_TH.drop(columns=['mssv', 'TH'])
y_TH = train_df_TH['TH']
search_model(X_TH, y_TH, models)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
#1 Model: linear
Best parameters: {'scaler': MinMaxScaler()}
Mean cross-validation score: 0.3309704715597821
------------------------------
Fitting 5 folds for each of 24 candidates, totalling 120 fits
#2 Model: ridge
Best parameters: {'regressor__alpha': 2.5, 'scaler': MinMaxScaler()}
Mean cross-validation score: 0.44177140260514564
------------------------------
Fitting 5 folds for each of 15 candidates, totalling 75 fits
#3 Model: lasso
Best parameters: {'regressor__alpha': 0.2, 'scaler': StandardScaler()}
Mean cross-validation score: 0.3175444657069201
------------------------------
Fitting 5 folds for each of 120 candidates, totalling 600 fits
#4 Model: elastic_net
Best parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.4, 'scaler': StandardScaler()}
Mean cross-validation score: 0.43261285335067
------------------------------
Fitting 5 folds for each of 105 candidates, totalling 525 fits
#5 Model: svr
Best par

[Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', LinearRegression())]),
 Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=2.5))]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor', Lasso(alpha=0.2, max_iter=500000))]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor',
                  ElasticNet(alpha=0.1, l1_ratio=0.4, max_iter=500000))]),
 Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=7.0, epsilon=0.4))]),
 Pipeline(steps=[('scaler', None),
                 ('regressor',
                  AdaBoostRegressor(learning_rate=0.03, n_estimators=300,
                                    random_state=123456))])]

In [ ]:
model_TH1 = Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=2.5))])
model_TH1.fit(X_TH, y_TH)

model_TH2 = Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=7.0, epsilon=0.4))])
model_TH2.fit(X_TH, y_TH)

list_model_TH = [model_TH1, model_TH2]
list_weight_TH = [0.5, 0.5]
ensemble_model_TH = WeightedVotingEnsemble(models=list_model_TH, weights=list_weight_TH)

Tìm mô hình và tham số tốt nhất cho việc dự đoán điểm cuối kỳ

In [ ]:
X_CK = train_df_CK.drop(columns=['mssv', 'CK'])
y_CK = train_df_CK['CK']
search_model(X_CK, y_CK, models)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
#1 Model: linear
Best parameters: {'scaler': MinMaxScaler()}
Mean cross-validation score: 0.21538883135179435
------------------------------
Fitting 5 folds for each of 24 candidates, totalling 120 fits
#2 Model: ridge
Best parameters: {'regressor__alpha': 4, 'scaler': MinMaxScaler()}
Mean cross-validation score: 0.314216284545551
------------------------------
Fitting 5 folds for each of 15 candidates, totalling 75 fits
#3 Model: lasso
Best parameters: {'regressor__alpha': 0.2, 'scaler': None}
Mean cross-validation score: 0.2436025409261017
------------------------------
Fitting 5 folds for each of 120 candidates, totalling 600 fits
#4 Model: elastic_net
Best parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.4, 'scaler': StandardScaler()}
Mean cross-validation score: 0.3029446842042781
------------------------------
Fitting 5 folds for each of 105 candidates, totalling 525 fits
#5 Model: svr
Best parameters: {'re

[Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', LinearRegression())]),
 Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=4))]),
 Pipeline(steps=[('scaler', None),
                 ('regressor', Lasso(alpha=0.2, max_iter=500000))]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('regressor',
                  ElasticNet(alpha=0.1, l1_ratio=0.4, max_iter=500000))]),
 Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=3.0, epsilon=0.6))]),
 Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor',
                  AdaBoostRegressor(learning_rate=0.1, n_estimators=200,
                                    random_state=123456))])]

In [ ]:
model_CK1 = Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=4))])
model_CK1.fit(X_CK, y_CK)

model_CK2 = Pipeline(steps=[('scaler', MinMaxScaler()),
                 ('regressor', SVR(C=3.0, epsilon=0.6))])
model_CK2.fit(X_CK, y_CK)

list_model_CK = [model_CK1, model_CK2]
list_weight_CK = [0.5, 0.5]
ensemble_model_CK = WeightedVotingEnsemble(models=list_model_CK, weights=list_weight_CK)

## Predict

In [ ]:
def predict(model, df, predict_col):
    temp_df = df.copy()
    results = model.predict(temp_df.drop(columns=['mssv']))
    temp_df['result'] = results
    temp_df['result'] = temp_df['result'].apply(lambda x: min(max(x, 0), 10))
    result_df = temp_df[['mssv', 'result']]
    result_df.to_csv(f'out_{predict_col}_final.csv', index=False, header=False)

predict(ensemble_model_QT, predict_df_QT, 'QT')

predict(ensemble_model_TH, predict_df_QT, 'TH')

predict(ensemble_model_CK, predict_df_CK, 'CK')

predict(ensemble_model_TBTL, predict_df_TBTL, 'TBTL')